In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils import data
import matplotlib.gridspec as gridspec

### Hyperparameters

In [ ]:
tau = 1
output_sizes = [3,2]

# Batch size for Stochastic Gradient descent
batch_size = 10000

# How many hidden layers the network chi has
network_depth = 3

# Width of every layer of chi
layer_width = 30

# Learning rate used for the ADAM optimizer

# create a list with the number of nodes for each layer
nodes = [layer_width]*network_depth

number_subsystems = len(output_sizes)

# How strong the fake subsystem is
factor_fake = 0.
# How large the noise in the mask for regularization is
noise = 1.
# Threshold after which the attention weight is set to zero
cutoff=0.9
# Learning rate
learning_rate=0.005
# epsilon
epsilon=1e-8
# score method
score_mode='regularize' # one of ('trunc', 'regularize', 'clamp', 'old')

### Create toymodel

In [ ]:
from examples import Toymodel_2Systems

In [ ]:
eps_list = [.025, .125, .05, .1]
toymodel = Toymodel_2Systems(eps_list)

### Sample hidden and observable trajectory

In [ ]:
# training data with 100000 steps
hidden_state_traj, observable_traj = toymodel.generate_traj(100000)

# validation data with 10000 steps
hidden_state_traj_valid, observable_traj_valid = toymodel.generate_traj(10000)

### Plot trajectory and true global eigenfunctions

In [ ]:
toymodel.plot_toymodel(hidden_state_traj_valid, observable_traj_valid)

In [ ]:
toymodel.plot_eigfunc(hidden_state_traj_valid, observable_traj_valid)

### Define training and validation set

In [ ]:
from deeptime.util.data import TrajectoryDataset

train_data = TrajectoryDataset(lagtime=tau, trajectory=observable_traj.astype('float32'))
val_data = TrajectoryDataset(lagtime=tau, trajectory=observable_traj_valid.astype('float32'))

### Define networks

In [ ]:
from masks import Mask
from collections import OrderedDict
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
input_size = observable_traj.shape[1] 
mask = Mask(input_size, number_subsystems, mean=torch.Tensor(train_data.data.mean(0)),
            std=torch.Tensor(train_data.data.std(0)), factor_fake=factor_fake, noise=noise, 
            device=device, cutoff=cutoff)
mask.to(device=device)
lobes = []
for output_size in output_sizes:
    lobe_dict = OrderedDict([('Layer_input', nn.Linear(input_size, layer_width)),
                            ('Elu_input', nn.ELU())])
    for d in range(network_depth):
        lobe_dict['Layer'+str(d)]=nn.Linear(layer_width, layer_width)
        lobe_dict['Elu'+str(d)]=nn.ELU()
    lobe_dict['Layer_output']=nn.Linear(layer_width, output_size)
    lobe_dict['Softmax']=nn.Softmax(dim=1) # obtain fuzzy probability distribution over output states
    
    lobe = nn.Sequential(
        lobe_dict 
    )
    lobes.append(lobe.to(device=device))

print(mask)
print(lobes)
                 

### Create iVAMPnets estimator

In [ ]:
from ivampnets import iVAMPnet

In [ ]:
ivampnet = iVAMPnet(lobes, mask, device, learning_rate=learning_rate, epsilon=epsilon, score_mode=score_mode)

### Plot mask before training

In [ ]:
from examples import plot_mask
plot_mask(mask)

### Create data loader

In [ ]:
from torch.utils.data import DataLoader

loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(val_data, batch_size=len(val_data), shuffle=False)

### Create a tensorboard writer to observe performance during training

In [ ]:
tensorboard_installed = False
if tensorboard_installed:
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('./runs/Toy2/')
    input_model, _ = next(iter(loader_train))
else:
    writer=None

### Fit the model on the training data

In [ ]:
model = ivampnet.fit(loader_train, n_epochs=20, validation_loader=loader_val, mask=True, lam_decomp=0., 
                     lam_trace=1., start_mask=0, end_trace=2, tb_writer=writer, clip=False).fetch_model()
if ivampnet.train_pen_scores[-1,1]>0.02:
    print('The model does not seem to be converged to an independent solution!')

In [ ]:
# execution time (cpu): ~ 30 sec

### Plot the training and validation scores

In [ ]:
plt.loglog(*ivampnet.train_scores.T, label='training')
plt.loglog(*ivampnet.validation_scores.T, label='validation')
plt.xlabel('step')
plt.ylabel('score')
plt.legend();

### Plot the mask after the training

In [ ]:
plot_mask(mask)

In [ ]:
# reproduces Fig. 3b (or permutation of it)

### Compare the eigenvalues from the true and the estimated transition matrix

In [ ]:
mask.noise=0 # set the noise to zero. 
T_list = model.get_transition_matrix(val_data.data, val_data.data_lagged)

In [ ]:
# Estimated eigenvalues
for T in T_list:
    print(np.linalg.eigvals(T))

In [ ]:
# True eigenvalues
print(np.linalg.eigvals(toymodel.T1)), np.linalg.eigvals(toymodel.T2)

### Plot the state assignment

In [ ]:
from examples import plot_states
plot_states(model, val_data.data)

### Estimate the eigenfunctions

In [ ]:
from examples import plot_eigfuncs
plot_eigfuncs(model, val_data)

In [ ]:
# reproduces Fig. 3c (possibly with permutation of state assignments)